In [ ]:
pip install konlpy

     |████████████████████████████████| 19.4 MB 5.3 MB/s 
     |████████████████████████████████| 448 kB 55.2 MB/s 
     |████████████████████████████████| 86 kB 4.9 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
from konlpy.tag import Mecab

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.manifold import TSNE
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def make_doc2vec_models(tagged_data, vector_size=128, window = 3, epochs = 40, min_counts = 0, workers = 4):
  model = Doc2Vec(tagged_data, vector_size = vector_size, window = window, epochs = epochs, min_counts = min_counts, workers=workers)
  return model

#파이선 doc2vec 모델 학습, 128차원 고정 길이의 벡터로 만듦

def make_doc2vec_webtoon(data, column, t_document=False):
    data_doc = []
    for tag, doc in zip(data.index, data[column]):
        doc = doc.split(" ")
        data_doc.append(([tag], doc))
    if t_document:
        data = [TaggedDocument(words=text, tags=tag) for tag, text in data_doc]
        return data
    else:
        return data_doc

#doc2vec에서 사용할 수 있는 데이터 생성
#doc2vec model 학습에 사용됩니다.

def recommendation_contents(user, data_doc, model):
  scores = []

  for tags, text in data_doc:
    train_doc_vec = model.docvecs[tags[0]]
    scores.append(cosine_similarity(user.reshape(-1,128), train_doc_vec.reshape(-1,128)))

  scores = np.array(scores).reshape(-1)
  scores = np.argsort(-scores)[:5]

  return webtoon.loc[scores, :]

#추천 결과 반환, 상위 N개를 추천, cosine similarity가 높은 인덱스를 추출해서 가져옵니다.

def make_user_embedding(index_list, data_doc, model):
    user = []
    user_embedding = []
    for i in index_list:
        user.append(data_doc[i][0][0])
    for i in user:
        user_embedding.append(model.docvecs[i])
    user_embedding = np.array(user_embedding)
    user = np.mean(user_embedding, axis = 0)
    return user

#user embedding 생성
#content vector기반의 user history를 평균해서 user embedding을 만들어주는 함수

def user_history(data):
    print(data[['title']])

#사용자의 히스토리 출력

In [ ]:
from google.colab import files
myfile = files.upload()

Saving webtoon_processing.csv to webtoon_processing.csv


In [ ]:
webtoon = pd.read_csv('webtoon_processing.csv')
webtoon.head()

,Unnamed: 0,title,genre,title_genre_story,mecab
0,0,신의 탑,판타지,신의 탑 판타지 자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년그리고 그런 소년...,"['신', '탑', '판타지', '자신', '모든', '것', '었', '던', '..."
1,1,참교육,액션,참교육 액션 무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다부...,"['참', '교육', '액션', '무너진', '교권', '을', '지키', '기',..."
2,2,뷰티풀 군바리,드라마,뷰티풀 군바리 드라마 여자도 군대에 간다면본격 여자도 군대 가는 만화,"['뷰티풀', '군바리', '드라마', '여자', '군대', '간다면', '본격',..."
3,3,윈드브레이커,스포츠,윈드브레이커 스포츠 혼자서 자전거를 즐겨타던 모범생 조자현원치 않게 자전거 크루의 ...,"['윈드브레이커', '스포츠', '혼자', '서', '자전거', '즐겨', '타',..."
4,4,장씨세가 호위무사,액션,장씨세가 호위무사 액션 당신이 부른 것이오 나란 사람을 은둔고수 광휘 호위무사 되다...,"['장', '씨세', '호위', '무사', '액션', '당신', '부른', '것',..."


In [ ]:
webtoon_doc_title_content_tag = make_doc2vec_webtoon(webtoon,'title_genre_story', t_document=True)
#print(webtoon_doc_title_content_tag)
webtoon_doc_title_content = make_doc2vec_webtoon(webtoon, 'title_genre_story')
#print(webtoon_doc_title_content)
webtoon_doc_tok_tag = make_doc2vec_webtoon(webtoon, 'mecab', t_document=True)
#print(webtoon_doc_tok_tag)
webtoon_doc_tok = make_doc2vec_webtoon(webtoon, 'mecab')
#print(webtoon_doc_tok)
#tag가 붙은 데이터는 doc2vec model 학습에 사용되며, 없는 데이터는 user embedding, cosine similarity를 구할 떄 사용됩니다.

In [ ]:
model_title_genre_story = make_doc2vec_models(webtoon_doc_title_content_tag)
model_tok = make_doc2vec_models(webtoon_doc_tok_tag)

In [ ]:
user1 = webtoon.loc[random.sample(webtoon.loc[webtoon.genre == '액션', :].index.values.tolist(),5),:]
user_history(user1)
#user1,2,3 각 카테고리별로 user history를 만드는 과정

      title
66    한림체육관
275  히어로 킬러
83      몬스터
74      헬58
392   천하제일인


In [ ]:
user2 = webtoon.loc[random.sample(webtoon.loc[webtoon.genre == '스릴러', :].index.values.tolist(),5),:]
user_history(user2)

               title
17              꼬리잡기
219  뜨거운 양철지붕 위의 고양이
229             로그아웃
104       나는 어디에나 있다
196        묵시의 인플루언서


In [ ]:
user3 = webtoon.loc[random.sample(webtoon.loc[webtoon.genre == '판타지', :].index.values.tolist(),5),:]
user_history(user3)

       title
218    마법사랑해
305      트럼프
95    3cm 헌터
60   오로지 오로라
11      만렙돌파


In [ ]:
user_action = make_user_embedding(user1.index.values.tolist(), webtoon_doc_title_content,model_title_genre_story)
user_thriller = make_user_embedding(user2.index.values.tolist(), webtoon_doc_title_content, model_title_genre_story)
user_fantasy = make_user_embedding(user3.index.values.tolist(), webtoon_doc_title_content, model_title_genre_story)
#각 장르마다 embedding을 통해서 128개의 평균을 구하게 됩니다.

In [ ]:
result = recommendation_contents(user_action, webtoon_doc_title_content, model_title_genre_story)
pd.DataFrame(result.loc[:, ['title','genre', 'title_genre_story']])

,title,genre,title_genre_story
396,나만 보여!,드라마,나만 보여! 드라마 학교 익명게시판이 나에게만 실명으로 보인다면교내 익명 어플 몰래...
56,개밥 먹는 남자,판타지,개밥 먹는 남자 판타지 출근길에 우연히 받은 아임펫 향수를 뿌렸다가 댕댕이 뚜치와 ...
129,피로만땅,개그,피로만땅 개그 호 저승사자 샤니 작가 신작 취준생 유림은 밤 시에 면접을 보러오라는...
39,사랑의 헌옷수거함,로맨스,사랑의 헌옷수거함 로맨스 짝사랑은 이제 그만 고잉고등학교에서 가장 핫한 소문 헌옷수...
275,히어로 킬러,액션,히어로 킬러 액션 전쟁은 빌런들의 패배로 막을 내렸다이제 세상은 부패한 히어로들이 ...


In [ ]:
result = recommendation_contents(user_thriller, webtoon_doc_title_content, model_title_genre_story)
pd.DataFrame(result.loc[:, ['title','genre', 'title_genre_story']])

,title,genre,title_genre_story
366,키스 식스 센스,로맨스,키스 식스 센스 로맨스 키스하는 순간 미래가 보인다키스하면 미래가 보이는 예술은 실...
17,꼬리잡기,스릴러,꼬리잡기 스릴러 건물붕괴 사고로 매몰된 명의 대학생주 뒤 명은 사망하고 명은 의식불...
256,나 혼자 만렙 뉴비,판타지,나 혼자 만렙 뉴비 판타지 게임 너튜버로 활동 중인 주인공 진혁은 유일하게 시련의 ...
396,나만 보여!,드라마,나만 보여! 드라마 학교 익명게시판이 나에게만 실명으로 보인다면교내 익명 어플 몰래...
211,흑막 여주가 날 새엄마로 만들려고 해,로맨스,흑막 여주가 날 새엄마로 만들려고 해 로맨스 친구들과 장난삼아 쓴 피폐 소설 속에 ...


In [ ]:
result = recommendation_contents(user_fantasy, webtoon_doc_title_content, model_title_genre_story)
pd.DataFrame(result.loc[:, ['title','genre', 'title_genre_story']])

,title,genre,title_genre_story
60,오로지 오로라,판타지,오로지 오로라 판타지 마법 소녀가 불법인 세상 이 세계의 마법 소녀는 사랑받는 존재...
435,제타,스릴러,제타 스릴러 극심한 슬럼프에 안드로이드 제타의 그림을 표절해버린 금세기 최고의 천재...
366,키스 식스 센스,로맨스,키스 식스 센스 로맨스 키스하는 순간 미래가 보인다키스하면 미래가 보이는 예술은 실...
211,흑막 여주가 날 새엄마로 만들려고 해,로맨스,흑막 여주가 날 새엄마로 만들려고 해 로맨스 친구들과 장난삼아 쓴 피폐 소설 속에 ...
129,피로만땅,개그,피로만땅 개그 호 저승사자 샤니 작가 신작 취준생 유림은 밤 시에 면접을 보러오라는...
